# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 18 2022 8:30AM,244413,10,0085940778,ISDIN Corporation,Released
1,Aug 18 2022 8:30AM,244413,10,0085940782,ISDIN Corporation,Released
2,Aug 18 2022 8:30AM,244413,10,0085940783,ISDIN Corporation,Released
3,Aug 18 2022 8:30AM,244413,10,0085940792,ISDIN Corporation,Released
4,Aug 18 2022 8:30AM,244413,10,0085940822,ISDIN Corporation,Released
5,Aug 18 2022 8:30AM,244416,10,0085940779,ISDIN Corporation,Released
6,Aug 18 2022 8:30AM,244416,10,0085940781,ISDIN Corporation,Released
7,Aug 18 2022 8:30AM,244416,10,0085940780,ISDIN Corporation,Released
8,Aug 18 2022 8:30AM,244416,10,0085940784,ISDIN Corporation,Released
9,Aug 18 2022 8:30AM,244416,10,0085940789,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
53,244413,Released,5
54,244415,Released,1
55,244416,Released,31
56,244417,Released,25
57,244418,Released,10


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244413,NaN,NaN,5.0
244415,NaN,NaN,1.0
244416,NaN,NaN,31.0
244417,NaN,NaN,25.0
244418,NaN,NaN,10.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244307,0.0,0.0,5.0
244312,4.0,5.0,12.0
244313,0.0,0.0,2.0
244315,0.0,0.0,1.0
244316,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244307,0,0,5
244312,4,5,12
244313,0,0,2
244315,0,0,1
244316,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244307,0,0,5
1,244312,4,5,12
2,244313,0,0,2
3,244315,0,0,1
4,244316,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244307,,,5
1,244312,4,5,12
2,244313,,,2
3,244315,,,1
4,244316,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 18 2022 8:30AM,244413,10,ISDIN Corporation
5,Aug 18 2022 8:30AM,244416,10,ISDIN Corporation
36,Aug 17 2022 3:46PM,244418,16,Sartorius Bioprocess Solutions
39,Aug 17 2022 3:46PM,244418,16,Sartorius Lab Products and Service
46,Aug 17 2022 3:43PM,244417,19,"AdvaGen Pharma, Ltd"
71,Aug 17 2022 3:35PM,244415,19,"NAPP Technologies, LLC"
72,Aug 17 2022 3:13PM,244410,10,AX Pharmaceutical Corp.
73,Aug 17 2022 3:12PM,244409,19,"AdvaGen Pharma, Ltd"
80,Aug 17 2022 2:42PM,244404,20,"ACI Healthcare USA, Inc."
81,Aug 17 2022 2:37PM,244406,17,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 18 2022 8:30AM,244413,10,ISDIN Corporation,,,5
1,Aug 18 2022 8:30AM,244416,10,ISDIN Corporation,,,31
2,Aug 17 2022 3:46PM,244418,16,Sartorius Bioprocess Solutions,,,10
3,Aug 17 2022 3:46PM,244418,16,Sartorius Lab Products and Service,,,10
4,Aug 17 2022 3:43PM,244417,19,"AdvaGen Pharma, Ltd",,,25
5,Aug 17 2022 3:35PM,244415,19,"NAPP Technologies, LLC",,,1
6,Aug 17 2022 3:13PM,244410,10,AX Pharmaceutical Corp.,,,1
7,Aug 17 2022 3:12PM,244409,19,"AdvaGen Pharma, Ltd",,3,4
8,Aug 17 2022 2:42PM,244404,20,"ACI Healthcare USA, Inc.",,,1
9,Aug 17 2022 2:37PM,244406,17,"AdvaGen Pharma, Ltd",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 8:30AM,244413,10,ISDIN Corporation,5,,
1,Aug 18 2022 8:30AM,244416,10,ISDIN Corporation,31,,
2,Aug 17 2022 3:46PM,244418,16,Sartorius Bioprocess Solutions,10,,
3,Aug 17 2022 3:46PM,244418,16,Sartorius Lab Products and Service,10,,
4,Aug 17 2022 3:43PM,244417,19,"AdvaGen Pharma, Ltd",25,,
5,Aug 17 2022 3:35PM,244415,19,"NAPP Technologies, LLC",1,,
6,Aug 17 2022 3:13PM,244410,10,AX Pharmaceutical Corp.,1,,
7,Aug 17 2022 3:12PM,244409,19,"AdvaGen Pharma, Ltd",4,3,
8,Aug 17 2022 2:42PM,244404,20,"ACI Healthcare USA, Inc.",1,,
9,Aug 17 2022 2:37PM,244406,17,"AdvaGen Pharma, Ltd",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 8:30AM,244413,10,ISDIN Corporation,5,,
1,Aug 18 2022 8:30AM,244416,10,ISDIN Corporation,31,,
2,Aug 17 2022 3:46PM,244418,16,Sartorius Bioprocess Solutions,10,,
3,Aug 17 2022 3:46PM,244418,16,Sartorius Lab Products and Service,10,,
4,Aug 17 2022 3:43PM,244417,19,"AdvaGen Pharma, Ltd",25,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 8:30AM,244413,10,ISDIN Corporation,5.0,NaN,NaN
1,Aug 18 2022 8:30AM,244416,10,ISDIN Corporation,31.0,NaN,NaN
2,Aug 17 2022 3:46PM,244418,16,Sartorius Bioprocess Solutions,10.0,NaN,NaN
3,Aug 17 2022 3:46PM,244418,16,Sartorius Lab Products and Service,10.0,NaN,NaN
4,Aug 17 2022 3:43PM,244417,19,"AdvaGen Pharma, Ltd",25.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 8:30AM,244413,10,ISDIN Corporation,5.0,0.0,0.0
1,Aug 18 2022 8:30AM,244416,10,ISDIN Corporation,31.0,0.0,0.0
2,Aug 17 2022 3:46PM,244418,16,Sartorius Bioprocess Solutions,10.0,0.0,0.0
3,Aug 17 2022 3:46PM,244418,16,Sartorius Lab Products and Service,10.0,0.0,0.0
4,Aug 17 2022 3:43PM,244417,19,"AdvaGen Pharma, Ltd",25.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5864801,211.0,10.0,8.0
102,244320,1.0,0.0,0.0
12,244391,1.0,0.0,0.0
15,244355,2.0,2.0,21.0
16,488836,20.0,0.0,0.0
17,488732,2.0,0.0,0.0
19,977626,33.0,15.0,0.0
20,1710549,119.0,7.0,0.0
21,977442,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5864801,211.0,10.0,8.0
1,102,244320,1.0,0.0,0.0
2,12,244391,1.0,0.0,0.0
3,15,244355,2.0,2.0,21.0
4,16,488836,20.0,0.0,0.0
5,17,488732,2.0,0.0,0.0
6,19,977626,33.0,15.0,0.0
7,20,1710549,119.0,7.0,0.0
8,21,977442,4.0,0.0,0.0
9,22,733133,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,211.0,10.0,8.0
1,102,1.0,0.0,0.0
2,12,1.0,0.0,0.0
3,15,2.0,2.0,21.0
4,16,20.0,0.0,0.0
5,17,2.0,0.0,0.0
6,19,33.0,15.0,0.0
7,20,119.0,7.0,0.0
8,21,4.0,0.0,0.0
9,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,211.0
1,102,Released,1.0
2,12,Released,1.0
3,15,Released,2.0
4,16,Released,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,17,19,20,21,22
Status,,,,,,,,,,
Completed,8.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,10.0,0.0,0.0,2.0,0.0,0.0,15.0,7.0,0.0,0.0
Released,211.0,1.0,1.0,2.0,20.0,2.0,33.0,119.0,4.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,17,19,20,21,22
0,Completed,8.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,0.0,0.0,2.0,0.0,0.0,15.0,7.0,0.0,0.0
2,Released,211.0,1.0,1.0,2.0,20.0,2.0,33.0,119.0,4.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,17,19,20,21,22
0,Completed,8.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,0.0,0.0,2.0,0.0,0.0,15.0,7.0,0.0,0.0
2,Released,211.0,1.0,1.0,2.0,20.0,2.0,33.0,119.0,4.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()